各種インポート及び環境変数の読み込み。

In [ ]:
import os
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import CodeInterpreterTool
from azure.identity import AzureCliCredential
from typing import Any
from pathlib import Path
from dotenv import load_dotenv
from IPython.display import Image, display

load_dotenv()

print(os.environ["PROJECT_CONNECTION_STRING"])

Agent のクライアントの作成

In [ ]:
project_client = AIProjectClient.from_connection_string(
    credential=AzureCliCredential(), conn_str=os.environ["PROJECT_CONNECTION_STRING"]
)
# 通常は with 文を使うのが良いが、jpyter notebook では使えないので明示的に __enter__ を呼び出す
project_client.__enter__()

Code Interpreter tool を使用する AI Agent の作成

In [ ]:
code_interpreter = CodeInterpreterTool()
# The CodeInterpreterTool needs to be included in creation of the agent
agent = project_client.agents.create_agent(
    model="gpt-4o-mini",
    name="my-agent",
    instructions="You are helpful agent",
    tools=code_interpreter.definitions,
    tool_resources=code_interpreter.resources,
)
print(f"Created agent, agent ID: {agent.id}")

会話のスレッドを作成

In [ ]:
# Create a thread
thread = project_client.agents.create_thread()
print(f"Created thread, thread ID: {thread.id}")


メッセージを作成してエージェントの実行

In [ ]:
# Create a message
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="""
    以下のデータを使って、営業利益の棒グラフを作成し、ファイルを提供してください。
    会社A: 1億2000万円, 会社B: 2億5000万円, 会社C: 3億円, 会社D: 1億8000万円
    棒グラフ内のテキストは会社名や通貨の表記も全て英語で生成してください。
    """,
)
print(f"Created message, message ID: {message.id}")

# Run the agent
run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
print(f"Run finished with status: {run.status}")


メッセージの表示

In [ ]:
# Get messages from the thread
messages = project_client.agents.list_messages(thread_id=thread.id)
print(f"Messages: {messages}")

# Get the last message from the sender
last_msg = messages.get_last_text_message_by_sender("assistant")
if last_msg:
    print(f"Last Message: {last_msg.text.value}")

# Generate an image file for the bar chart
for image_content in messages.image_contents:
    print(f"Image File ID: {image_content.image_file.file_id}")
    file_name = f"{image_content.image_file.file_id}_image_file.png"
    project_client.agents.save_file(file_id=image_content.image_file.file_id, file_name=file_name)
    print(f"Saved image file to: {Path.cwd() / file_name}")

ファイルのダウンロードと表示

In [ ]:
# Print the file path(s) from the messages
for file_path_annotation in messages.file_path_annotations:
    print(f"File Paths:")
    print(f"Type: {file_path_annotation.type}")
    print(f"Text: {file_path_annotation.text}")
    print(f"File ID: {file_path_annotation.file_path.file_id}")
    print(f"Start Index: {file_path_annotation.start_index}")
    print(f"End Index: {file_path_annotation.end_index}")
    project_client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=Path(file_path_annotation.text).name)
    display(Image(filename=Path(file_path_annotation.text).name))


In [ ]:
project_client.agents.delete_agent(agent.id)
# リソースの解放
project_client.close()
print("Deleted agent")